In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import joblib
import math
import numpy as np
import itertools 
from itertools import product
from collections import Counter
import garpar as gp
import re
import skcriteria as skc
from skcriteria import pipeline
from skcriteria.preprocessing import invert_objectives, weighters, scalers
from skcriteria.madm import similarity, moora, electre
from skcriteria.ranksrev import RankInvariantChecker
from garpar.datasets import make_multisector, RissoLevyStable, RissoNormal, RissoUniform
import warnings

In [38]:
por_multisector = gp.datasets.make_multisector(
    RissoLevyStable(price=200, alpha=1.6411, beta=-0.0126, mu=0.0005, sigma=0.005,   entropy=0.5, random_state=0), 
    RissoLevyStable(alpha=2, beta=-0.5, mu=0.0005, sigma=0.005,   entropy=0.5, random_state=100), 
    RissoLevyStable(alpha=1.90, beta=0.10, mu=0.001, sigma=0.004,   entropy=0.6, random_state=200),
    stocks=3, days=7, window_size=3)
por_multisector

TypeError: RissoLevyStable.__init__() got an unexpected keyword argument 'price'

In [70]:
#--- 1. Configuración de Parámetros Fijos (CORREGIDO) ---
NUM_ITERACIONES = 100
STOCKS_TOTALES = 6  # 💡 CAMBIO CLAVE: Un solo stock por sector
DIAS_SIMULACION = 7
VENTANAS_A_PROBAR = [3, 5, 7] 

# --- 2. Definición de Parámetros y PRECIOS PERSONALIZADOS (CORREGIDO) ---
PRECIOS_INICIALES_TOTAL = [
    200.0,  # alua
    300.0,  # Precio para el Stock 2 (Sector 2)
    150.0,
    100,
    100,
    800# Precio para el Stock 3 (Sector 3)
] 

# --- 3. Definición de Parámetros de Sectores (Sin Cambios) ---
params_A = {"alpha": 1.6411, "beta": -0.0126, "mu": 0.0005, "sigma": 0.005, "entropy": 0.5}
params_B = {"alpha": 2, "beta": -0.5, "mu": 0.0005, "sigma": 0.005, "entropy": 0.5}
params_C = {"alpha": 1.9, "beta": 0.1, "mu": 0.001, "sigma": 0.004, "entropy": 0.6}
params_D = {"alpha": 1.6411, "beta": -0.0126, "mu": 0.0005, "sigma": 0.005, "entropy": 0.5}
params_E = {"alpha": 1.6411, "beta": -0.0126, "mu": 0.0005, "sigma": 0.005, "entropy": 0.5}
params_F = {"alpha": 1.9, "beta": 0.1, "mu": 0.001, "sigma": 0.004, "entropy": 0.6}

parametros_sectores_base = [params_A, params_B, params_C, params_D, params_E]

# Diccionario para guardar todos los DataFrames: {Ventana: [df_corrida_1, df_corrida_2, ...]}
resultados_por_ventana = {ventana: [] for ventana in VENTANAS_A_PROBAR}

# --- 4. BUCLE DE SIMULACIÓN Y ALMACENAMIENTO ---

for VENTANA in VENTANAS_A_PROBAR:
    print(f"Iniciando {NUM_ITERACIONES} simulaciones para Ventana: {VENTANA} días...")
    
    for i in range(NUM_ITERACIONES):
        
        # Cálculo del random_state secuencial por bloque (i * 10000 + j * 100)
        random_seed_base = i * 10000 
        distribuciones_para_corrida = []
        
        for j, params in enumerate(parametros_sectores_base):
            semilla_sector = random_seed_base + (j * 100)
            params_corrida = params.copy() 
            params_corrida['random_state'] = semilla_sector
            distribuciones_para_corrida.append(RissoLevyStable(**params_corrida))

        # SIMULACIÓN DEL MERCADO
        stocksset = make_multisector(
            *distribuciones_para_corrida,
            stocks=STOCKS_TOTALES,
            days=DIAS_SIMULACION,
            window_size=VENTANA,
            price=PRECIOS_INICIALES_TOTAL 
        )
        
        # ALMACENAMIENTO DEL DATAFRAME DE PRECIOS
        df_simulacion = stocksset._prices_df 
        resultados_por_ventana[VENTANA].append(df_simulacion)
    
print("\n--- ¡Generación de Simulaciones Finalizada! ---")

Iniciando 100 simulaciones para Ventana: 3 días...
Iniciando 100 simulaciones para Ventana: 5 días...
Iniciando 100 simulaciones para Ventana: 7 días...

--- ¡Generación de Simulaciones Finalizada! ---


In [75]:
ms = gp.optimize.mean_variance.MVOptimizer( model='max_sharpe')
opt = ms.optimize(gp.StocksSet.from_prices(resultados_por_ventana[3][1])).weights_prune()
opt

/home/nadia/indicador/lib/python3.10/site-packages/garpar/optimize/mean_variance.py:259: UserWarning: No risk_free_rate specified, coercing it
  warnings.warn("No risk_free_rate specified, coercing it")


Stocks,"rissolevystable_1_S0[W 2.712158e-07, H?]","rissolevystable_1_S1[W 5.928203e-01, H?]","rissolevystable_2_S0[W 1.653358e-02, H?]","rissolevystable_3_S0[W 8.048122e-02, H?]","rissolevystable_4_S0[W 4.899525e-03, H?]","rissolevystable_5_S0[W 3.052651e-01, H?]"
Days,,,,,,
0,200.000000,300.000000,150.000000,100.000000,100.000000,800.000000
1,199.996419,299.992608,150.002110,100.004960,99.995847,800.006064
2,199.993191,299.998991,150.005913,99.997086,99.993960,800.003512
3,199.989050,300.000752,150.003696,99.997726,99.985207,800.010929
4,199.982728,300.006188,150.009023,100.002641,99.985154,800.021948
5,199.983832,300.009022,150.005420,100.003901,99.970578,800.020530
6,199.977323,300.013689,149.996476,100.001621,99.989780,800.020394
7,199.972734,300.011954,149.999369,100.000447,99.998616,800.010677


In [84]:
# --- 0. CONFIGURACIÓN INICIAL ---
# Se recomienda suprimir el warning común de garpar/pypfopt sobre matrices no positivas
warnings.filterwarnings('ignore', category=UserWarning, message='The covariance matrix is non positive semidefinite. Amending eigenvalues.')
warnings.filterwarnings('ignore', category=UserWarning, message="No risk_free_rate specified, coercing it")
# ⚠️ DEFINICIÓN DE LOS OPTIMIZADORES A PROBAR (Asumida)
optimizadores_a_probar = {
    "max_sharpe": gp.optimize.mean_variance.MVOptimizer(model='max_sharpe'),
    "markowitz_target_return": gp.optimize.mean_variance.Markowitz(target_return=0.001),
    "markowitz_target_risk": gp.optimize.mean_variance.Markowitz(target_risk=0.30),
}

rows = []

print("--- INICIO DEL PROCESAMIENTO DE MÉTRICAS (Usando StocksSet) ---")

# --- 1. BUCLE DE PROCESAMIENTO SOBRE LOS RESULTADOS GUARDADOS ---

for VENTANA, lista_simulaciones in resultados_por_ventana.items():
    
    for i, df_corrida in enumerate(lista_simulaciones):
        
        corrida_id = i + 1
        stocks_number_base = len(df_corrida.columns)

        # 💡 MEJORA DE ROBUSTEZ: Intentar crear el StocksSet primero
        stocksset_corrida = None
        try:
            stocksset_corrida = gp.StocksSet.from_prices(df_corrida, window_size=VENTANA)
        except Exception as e:
            # Si el StocksSet falla (ej: datos malformados), registra el fallo para TODOS los modelos
            print(f"⚠️ Error al crear StocksSet (Corrida {corrida_id}, Ventana {VENTANA}): {e}. Saltando modelos.")
            
            for opt_name in optimizadores_a_probar.keys():
                rows.append({
                    "Ventana": VENTANA,
                    "Corrida": corrida_id,
                    "Modelo_Opt": opt_name,
                    "stocks_number": stocks_number_base, 
                    "Cross_Entropy": np.nan,
                    "Zheng_Entropy": np.nan,
                    "Div_Ratio": np.nan,
                    "CV_MC": np.nan,
                    "PDI": np.nan
                })
            continue # Pasa a la siguiente corrida
        
        # 2. ITERACIÓN SOBRE LOS MODELOS DE OPTIMIZACIÓN
        for opt_name, optimizador in optimizadores_a_probar.items():
            
            try:
                # 3. OPTIMIZACIÓN + PRUNING
                mopt = optimizador.optimize(stocksset_corrida).weights_prune()
                
                # 4. CÁLCULO DE MÉTRICAS (EXITOSO)
                metricas = {
                    "stocks_number": len(mopt.stocks),
                    "Cross_Entropy": mopt.div.cross_entropy(),
                    "Zheng_Entropy": mopt.div.zheng_entropy(),
                    "Div_Ratio": mopt.div.ratio(),
                    "CV_MC": (mopt.div.mrc() * mopt.weights).std() / (mopt.div.mrc() * mopt.weights).mean(),
                    "PDI": mopt.div.pdi(n_components=3, random_state=100)
                }

                # 5. GUARDAR RESULTADO EXITOSO
                rows.append({
                    "Ventana": VENTANA,
                    "Corrida": corrida_id,
                    "Modelo_Opt": opt_name, 
                    **metricas
                })

            except Exception as e:
                # 🛠️ SOLUCIÓN AL KEYERROR: Garantizar que la fila de identificadores se guarde
                rows.append({
                    "Ventana": VENTANA,
                    "Corrida": corrida_id,
                    "Modelo_Opt": opt_name,
                    "stocks_number": stocks_number_base, 
                    "Cross_Entropy": np.nan,
                    "Zheng_Entropy": np.nan,
                    "Div_Ratio": np.nan,
                    "CV_MC": np.nan,
                    "PDI": np.nan
                })
                # print(f"⚠️ Error en {opt_name} (Corrida {corrida_id}, Ventana {VENTANA}): {e}") 
            
# --- 2. CREACIÓN DEL DATAFRAME FINAL ---

df_metricas_simulacion = pd.DataFrame(rows)

# Esta línea ahora es segura, ya que 'rows' garantiza las claves de índice
df_metricas_simulacion = df_metricas_simulacion.set_index(["Ventana", "Corrida", "Modelo_Opt"])

print("\n--- ANÁLISIS COMPLETADO ---")
print(f"Total de resultados guardados: {len(df_metricas_simulacion)} filas.")
print(df_metricas_simulacion.head(9))


--- INICIO DEL PROCESAMIENTO DE MÉTRICAS (Usando StocksSet) ---


/home/nadia/indicador/lib/python3.10/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/nadia/indicador/lib/python3.10/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/nadia/indicador/lib/python3.10/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/nadia/indicador/lib/python3.10/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/nadia/indicador/lib/python3.10


--- ANÁLISIS COMPLETADO ---
Total de resultados guardados: 900 filas.
                                         stocks_number  Cross_Entropy  \
Ventana Corrida Modelo_Opt                                              
3       1       max_sharpe                           5       4.913730   
                markowitz_target_return              6       0.119852   
                markowitz_target_risk                5      13.992721   
        2       max_sharpe                           6       3.003324   
                markowitz_target_return              6       1.930387   
                markowitz_target_risk                6      10.821468   
        3       max_sharpe                           6       9.529105   
                markowitz_target_return              6       0.523712   
                markowitz_target_risk                5      13.463153   

                                         Zheng_Entropy  Div_Ratio     CV_MC  \
Ventana Corrida Modelo_Opt                    

In [93]:
df_metricas_simulacion[df_metricas_simulacion.index.get_level_values('Ventana') == 3]

stocks_number  Cross_Entropy  \
Ventana Corrida Modelo_Opt                                              
3       1       max_sharpe                           5       4.913730   
                markowitz_target_return              6       0.119852   
                markowitz_target_risk                5      13.992721   
        2       max_sharpe                           6       3.003324   
                markowitz_target_return              6       1.930387   
...                                                ...            ...   
        99      markowitz_target_return              6       0.227400   
                markowitz_target_risk                6      10.997246   
        100     max_sharpe                           4       5.212613   
                markowitz_target_return              6       0.153063   
                markowitz_target_risk                6      13.722991   

                                         Zheng_Entropy  Div_Ratio     CV_MC  \
Ventana Corrida Modelo_Opt                                                    
3       1       max_sharpe                9.352603e-01   0.193362  2.433017   
                markowitz_target_return   1.673168e+00   0.293970  0.208312   
                markowitz_target_risk     2.885831e-07   0.062994  2.236068   
        2       max_sharpe                9.688648e-01   0.112811  1.994678   
                markowitz_target_return   3.510643e-01   0.072490  2.493270   
...                                                ...        ...       ...   
        99      markowitz_target_return   1.575403e+00   0.098049  1.573088   
                markowitz_target_risk     2.266696e-05   0.062994  2.449490   
        100     max_sharpe                4.239749e-01   0.128274  1.699337   
                markowitz_target_return   1.671602e+00   0.102438  1.011466   
                markowitz_target_risk     6.246258e-06   0.062994  2.449488   

                                              PDI  
Ventana Corrida Modelo_Opt                         
3       1       max_sharpe               1.789501  
                markowitz_target_return  1.748395  
                markowitz_target_risk    1.755222  
        2       max_sharpe               1.381961  
                markowitz_target_return  1.381961  
...                                           ...  
        99      markowitz_target_return  1.322926  
                markowitz_target_risk    1.322926  
        100     max_sharpe               1.662071  
                markowitz_target_return  1.661111  
                markowitz_target_risk    1.661111  

[300 rows x 6 columns]

In [91]:
Objectives = [
    max, #     'stocks_seleccionados', coincide con definición mayor número de stocks
    min, #     'Entropia_cruzada', mide el desorden al compararlo con una estrategia teórica de risk part equally
    max, #     'Entropia_Zheng', 
    max, #     'Ratio_diversific', mide la diversificación ganada por  mantener activos no correlacionados
    min, #     'CV_Contribuciones_marginales', si todos los activos tienen igual contribución marginal es un portfolio diversificado
    max, #     'Indice_diversific_port, si hay correlacióan alta el índice dará un valor bajo mientras que si no hay correlación el índice tenderá al valor N
]
dm = skc.mkdm(metrics.to_numpy(), objectives, alternatives=metrics.index, criteria=metrics.columns)

NameError: name 'metrics' is not defined

In [ ]:
pipe_igual_vector = pipeline.mkpipe(
    invert_objectives.NegateMinimize(),
    scalers.VectorScaler(target="matrix"),
    weighters.EqualWeighter(),
    similarity.TOPSIS(),
)
pipe_igual_vector
result_igual_vector = pipe_igual_vector.evaluate(dm)
result_igual_vector